<a href="https://colab.research.google.com/github/spps-supalerk/CRM_Analytics/blob/main/aDayInLife.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import matplotlib as mpl
plt.rcParams['font.family'] = 'tahoma'
!wget -q https://github.com/Phonbopit/sarabun-webfont/raw/master/fonts/thsarabunnew-webfont.ttf
mpl.font_manager.fontManager.addfont('thsarabunnew-webfont.ttf')
mpl.rc('font', family='TH Sarabun New', size=10)

In [2]:
df_day = pd.read_csv('/content/drive/MyDrive/CRM_Analytics/Assignment 3 - A Day in Life/adayinlife.csv')
df_day.head()

,time,activity,pain,gain,stu_id,time_type,time_hr,time_min
0,08.30,ตื่นนอน,ไม่อยากตื่น อยากนอนต่อ,ได้เริ่มต้นวันใหม่,6210422033,NaN,08,30
1,08.35,อาบน้ำ,ยังอยากนอนต่อ,สดชื่น+สะอาด,6210422033,NaN,08,35
2,09.00,กินอาหาเช้า,ต้องสั่ง Delivery,อิ่มท้อง,6210422033,NaN,09,0
3,10.00,เดินทางด้วย BTS ไปทำงาน,ราคาสูง,เดินทางสะดวกสบาย,6210422033,NaN,10,0
4,10.30,ถึงที่ทำงาน เริ่มทำงาน,NaN,เป็นช่วงที่พร้อมทำงานที่สุด,6210422033,NaN,10,30


In [3]:
# for those who want to use simple function --> use only {time_hr}.00-{time_hr+1}.00
def round_time(df, min_round = 60):
  hr = df['time_hr']
  min = df['time_min']
  hr += ( min //min_round )
  return f'{str(hr).zfill(2)}.00-{str(hr+1).zfill(2)}.00'

In [4]:
# clean time
df_day['time_hr'] = df_day['time_hr'].replace('24','0')
df_day['time_hr'] = df_day['time_hr'].astype(int)
df_day['time_min'] = df_day['time_min'].astype(int)
df_day['time_range'] = df_day.apply(round_time,axis=1)

# get activity number
df_day['activity_num'] = df_day.groupby(['stu_id']).cumcount()+1

In [5]:
df_day.head()

,time,activity,pain,gain,stu_id,time_type,time_hr,time_min,time_range,activity_num
0,08.30,ตื่นนอน,ไม่อยากตื่น อยากนอนต่อ,ได้เริ่มต้นวันใหม่,6210422033,NaN,8,30,08.00-09.00,1
1,08.35,อาบน้ำ,ยังอยากนอนต่อ,สดชื่น+สะอาด,6210422033,NaN,8,35,08.00-09.00,2
2,09.00,กินอาหาเช้า,ต้องสั่ง Delivery,อิ่มท้อง,6210422033,NaN,9,0,09.00-10.00,3
3,10.00,เดินทางด้วย BTS ไปทำงาน,ราคาสูง,เดินทางสะดวกสบาย,6210422033,NaN,10,0,10.00-11.00,4
4,10.30,ถึงที่ทำงาน เริ่มทำงาน,NaN,เป็นช่วงที่พร้อมทำงานที่สุด,6210422033,NaN,10,30,10.00-11.00,5


In [6]:
# flag last activity
df_last_act = df_day.groupby(['stu_id'])['activity_num'].max().reset_index()
df_last_act['last_act'] = 1

# merge last activity to df_day
df_day = df_day.merge(df_last_act, on= ['stu_id','activity_num'], how = 'left')

# when BADS student wake
df_wake = df_day[df_day['activity_num']==1][['stu_id','time_range']]
df_wake.columns = ['stu_id','wake_time']

# when BADS student sleep
df_sleep = df_day[df_day['last_act']==1][['stu_id','time_range']]
df_sleep.columns = ['stu_id','sleep_time']

In [7]:
# create df contains stu_id, awake_hr (number of awake time in hr), sleep_hr (number of sleep time in hr)
wake2sleep = df_wake.merge(df_sleep,how='inner', on = 'stu_id')
wake2sleep['wake_time_hr'] = wake2sleep['wake_time'].apply(lambda x:int(x.split('.')[0]))
wake2sleep['sleep_time_hr'] = wake2sleep['sleep_time'].apply(lambda x:int(x.split('.')[0]))
wake2sleep['awake_time'] = wake2sleep['sleep_time_hr'] - wake2sleep['wake_time_hr'] 
wake2sleep['awake_time'] = wake2sleep['awake_time'].apply(lambda x: x+24 if x<=0 else x)
wake2sleep['sleep_time'] = 24 - wake2sleep['awake_time'] 
wake2sleep = wake2sleep[['stu_id','awake_time','sleep_time']].rename(columns = {'awake_time':'awake_hr', 'sleep_time' : 'sleep_hr'})
wake2sleep.head()

,stu_id,awake_hr,sleep_hr
0,6210422033,16,8
1,6210422035,16,8
2,6220422004,18,6
3,6310422093,18,6
4,6310422096,17,7


In [16]:
# something wrong with 5
wake2sleep['awake_hr'].value_counts()

18    18
17    14
19     6
16     6
15     1
5      1
Name: awake_hr, dtype: int64

In [17]:
# something wrong with 19
wake2sleep['sleep_hr'].value_counts()

6     18
7     14
8      6
5      6
19     1
9      1
Name: sleep_hr, dtype: int64

In [15]:
df_day.head()

,time,activity,pain,gain,stu_id,time_type,time_hr,time_min,time_range,activity_num,last_act
0,08.30,ตื่นนอน,ไม่อยากตื่น อยากนอนต่อ,ได้เริ่มต้นวันใหม่,6210422033,NaN,8,30,08.00-09.00,1,NaN
1,08.35,อาบน้ำ,ยังอยากนอนต่อ,สดชื่น+สะอาด,6210422033,NaN,8,35,08.00-09.00,2,NaN
2,09.00,กินอาหาเช้า,ต้องสั่ง Delivery,อิ่มท้อง,6210422033,NaN,9,0,09.00-10.00,3,NaN
3,10.00,เดินทางด้วย BTS ไปทำงาน,ราคาสูง,เดินทางสะดวกสบาย,6210422033,NaN,10,0,10.00-11.00,4,NaN
4,10.30,ถึงที่ทำงาน เริ่มทำงาน,NaN,เป็นช่วงที่พร้อมทำงานที่สุด,6210422033,NaN,10,30,10.00-11.00,5,NaN


# Universal Sentence Encoder (USE)

In [18]:
%tensorflow_version 2.x

In [19]:
!pip install tensorflow_text

     |████████████████████████████████| 3.4MB 5.8MB/s 


In [20]:
import tensorflow_hub as hub
import tensorflow_text
import tensorflow as tf
import json
import itertools

In [21]:
#download open subtitle EN-TH dataset
!wget http://opus.nlpl.eu/download.php?f=OpenSubtitles/v2018/moses/en-th.txt.zip

--2021-01-30 13:37:59--  http://opus.nlpl.eu/download.php?f=OpenSubtitles/v2018/moses/en-th.txt.zip
Resolving opus.nlpl.eu (opus.nlpl.eu)... 193.166.25.9
Connecting to opus.nlpl.eu (opus.nlpl.eu)|193.166.25.9|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/moses/en-th.txt.zip [following]
--2021-01-30 13:38:00--  https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/moses/en-th.txt.zip
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.19, 86.50.254.18
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 119520928 (114M) [application/zip]
Saving to: ‘download.php?f=OpenSubtitles%2Fv2018%2Fmoses%2Fen-th.txt.zip’

download.php?f=Open 100%[===================>] 113.98M  22.5MB/s    in 5.9s    

2021-01-30 13:38:07 (19.3 MB/s) - ‘download.php?f=OpenSubtitles%2Fv2018%2Fmoses%2Fen-th.txt.zip’ saved [119520928/

In [22]:
!unzip download.php?f=OpenSubtitles%2Fv2018%2Fmoses%2Fen-th.txt.zip -d data

Archive:  download.php?f=OpenSubtitles%2Fv2018%2Fmoses%2Fen-th.txt.zip
  inflating: data/OpenSubtitles.en-th.en  
  inflating: data/OpenSubtitles.en-th.th  
  inflating: data/OpenSubtitles.en-th.ids  
  inflating: data/README             


In [23]:
#Create USE embedding by using tensorflow hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

In [24]:
df_pain = df_day['pain'].astype(str)
df_gain = df_day['gain'].astype(str)

df_use_pain = pd.DataFrame(embed(df_pain.values).numpy(),index= df_pain.values)
df_use_gain = pd.DataFrame(embed(df_gain.values).numpy(),index= df_gain.values)

## Cluster Free Text

In [ ]:
import umap
from sklearn.cluster import DBSCAN

In [ ]:
df_trans_pain = umap.UMAP(n_neighbors=5, random_state=42).fit_transform(df_use_pain)
df_trans_gain = umap.UMAP(n_neighbors=5, random_state=42).fit_transform(df_use_gain)

In [ ]:
# you can adjust eps as you want
label_pain = DBSCAN(eps=1, min_samples=3).fit_predict(df_trans_pain)
label_gain = DBSCAN(eps=1, min_samples=3).fit_predict(df_trans_gain)

In [ ]:
df_result_pain = pd.DataFrame({'pain' : df_pain,
                               'cluster' : label_pain})
df_result_gain = pd.DataFrame({'gain' : df_gain,
                               'cluster' : label_gain})

In [ ]:
# loop display each cluster number
unq_pain = df_result_pain['cluster'].unique()
unq_pain.sort()
for cluster in unq_pain:
  temp = df_result_pain[df_result_pain['cluster']==cluster]
  display(temp.shape)
  display(temp)

## Group Pain Cluster
<li> ความลำคาล : 0
<li> อยากนอน - ง่วง : 1 4 12
<li> แพง : 2
<li> nan : 3
<li> น้ำเย็น / หนาว : 5 7 11
<li> หิว / กลัวอ้วน : 6
<li> งานเพิ่ม / งานไม่เสร็จ : 8 13 16
<li> เหนื่อย : 9
<li> รถติด/ขับรถ : 10 15
<li> เบื่อ : 14

In [ ]:
dic_group_pain = {
    0 : 'ความลำคาญ',
    1 : 'ง่วง',
    4 : 'ง่วง',
    12 :'ง่วง',
    2 : 'แพง',
    3 : 'ไม่ระบุ',
    5 : 'หนาว',
    7 : 'หนาว',
    11 :'หนาว',
    6 : 'หิว-กลัวอ้วน',
    8 : 'งานเพิ่ม-ไม่เสร็จ', 
    13 : 'งานเพิ่ม-ไม่เสร็จ', 
    16 : 'งานเพิ่ม-ไม่เสร็จ',
    9 : 'เหนื่อย',
    10 : 'รถติด-ต้องขับรถ',
    15 : 'รถติด-ต้องขับรถ',
    14 : 'รถติด-ต้องขับรถ'
}

## Group Gain Cluster
<li> สดชื่น : 0 1 8 11 14
<li> อิ่ม : 2 9
<li> ฟิน : 3 20 19 18
<li> พักผ่อน : 4 6 16 17
<li> เรียนรู้สิ่งใหม่ : 5 12
<li> nan : 7
<li> ได้เงิน : 10
<li> ออกกำลังกาย : 13
<li> ทำการบ้าน : 15


In [ ]:
unq_gain = df_result_gain['cluster'].unique()
unq_gain.sort()
for cluster in unq_gain:
  temp = df_result_gain[df_result_gain['cluster']==cluster]
  display(temp.shape)
  display(temp)

In [ ]:
dic_group_gain = {
    0 : 'สดชื่น',
    1 : 'สดชื่น',
    8 : 'สดชื่น',
    11 :'สดชื่น',
    14 :'สดชื่น',
    2 : 'อิ่ม',
    9 : 'อิ่ม',
    3 : 'ฟิน',
    20 :'ฟิน',
    19 :'ฟิน',
    18 :'ฟิน',
    4 : 'พักผ่อน',
    6 : 'พักผ่อน',
    16 :'พักผ่อน',
    17 :'พักผ่อน',
    5 : 'เรียนรู้สิ่งใหม่',
    12 :'เรียนรู้สิ่งใหม่',
    7 : 'ไม่ระบุ',
    10 :'ได้เงิน',
    13 :'ออกกำลังกาย',
    15 :'ทำการบ้าน'
}

## Assign Gain & Pain Cluster/Main Category

In [ ]:
df_result_pain['group_pain'] = df_result_pain['cluster'].replace(dic_group_pain)
df_result_gain['group_gain'] = df_result_gain['cluster'].replace(dic_group_gain)
print('\n------------Group of Pain--------------\n')
display(df_result_pain['group_pain'].value_counts().head(8))
print('\n------------Group of Gain--------------\n')
display(df_result_gain['group_gain'].value_counts().head(8))

In [ ]:
df_day['group_pain'] = df_result_pain['cluster'].replace(dic_group_pain)
df_day['group_gain'] = df_result_gain['cluster'].replace(dic_group_gain)
df_day.head()

In [ ]:
df_day.head()

In [ ]:
# ignore ไม่ระบุ of group_pain
df_day_pain = df_day.groupby(['time_range','group_pain'])['stu_id'].count().reset_index()
df_day_pain = df_day_pain[df_day_pain['group_pain']!='ไม่ระบุ']
# ignore ไม่ระบุ of group_gain
df_day_gain = df_day.groupby(['time_range','group_gain'])['stu_id'].count().reset_index()
df_day_gain = df_day_gain[df_day_gain['group_gain']!='ไม่ระบุ']

In [ ]:
df_day_pain.head()

In [ ]:
df_day_gain

## Scatter plot

In [ ]:
fig, ax = plt.subplots(2,1,figsize =(20,18), sharex =True, sharey=True)
sns.scatterplot(x='time_range', y='stu_id', 
                data = df_day_pain, ax=ax[0], 
                s= 80, label = 'PAIN',
                color ='r')
sns.scatterplot(x='time_range', y='stu_id', data = df_day_gain, ax=ax[1], s= 80, label = 'GAIN', color='green')

for i in df_day_pain.index:
  ax[0].annotate(df_day_pain.loc[i,'group_pain'], (df_day_pain.loc[i,'time_range'], df_day_pain.loc[i,'stu_id']))

for i in df_day_gain.index:
  ax[1].annotate(df_day_gain.loc[i,'group_gain'], (df_day_gain.loc[i,'time_range'], df_day_gain.loc[i,'stu_id']))

ax[1].set_xticklabels(df_day_pain['time_range'].unique(),rotation=45)
plt.show()

## Write file

In [ ]:
df_day_gain.to_csv('daily_gain.csv',index=False)
df_day_pain.to_csv('daily_pain.csv',index=False)
df_day.to_csv('daily_all.csv', index=False)
wake2sleep.to_csv('wake2sleep.csv',index=False)

In [ ]:
# use for tree decomposition in power bi
df_pain_pv = df_day[df_day['group_pain']!='ไม่ระบุ'].pivot_table(index='stu_id', columns = 'activity_num',values = 'group_pain', aggfunc = 'sum')
df_gain_pv = df_day[df_day['group_gain']!='ไม่ระบุ'].pivot_table(index='stu_id', columns = 'activity_num',values = 'group_gain', aggfunc = 'sum')
df_pain_pv.to_csv('df_pain_pv.csv')
df_gain_pv.to_csv('df_gain_pv.csv')